In [14]:
%load_ext lab_black
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_hdf("../data/observations.hdf", "key")
df.obs_rarity = df.obs_rarity.clip(1, 1)
df = df.groupby(["BU_NAAM", "year"]).sum()
df = df[df.obs_rarity > 0].sort_values("year")
df = df.obs_rarity.unstack("year")

df_norm = df / df.sum()
df_norm = df_norm.T

df_norm_wijk = df.T / df.T.mean()

df_evi = pd.read_csv("../data/satdata/evi.csv", index_col=0)
df_evi["year"] = df_evi.index.str.slice(0, 4)
df_evi_year = df_evi.groupby(df_evi.year).mean()
df_evi_year.index = df_evi_year.index.astype(int)
df_evi_year_change = df_evi_year.ffill().pct_change()

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [15]:
# heel lelijk
res = []
for (
    year,
    row,
) in (
    df.T.iterrows()
):  # itereer over df_norm (normalized per year (/ sum year)) of over df.T
    for wijk in row.keys():
        try:
            evi_val = df_evi_year.loc[int(year)][wijk]
        except Exception:
            continue
        res.append({"year": year, "wijk": wijk, "obs": row[wijk], "evi": evi_val})
df_obs = pd.DataFrame(res)
# df_res.head()

In [25]:
evi = pd.read_csv(
    "../data/satdata/evi.csv", parse_dates=[0], index_col=[0]
).interpolate()
stemp = pd.read_csv(
    "../data/satdata/stemp.csv", parse_dates=[0], index_col=[0]
).interpolate()
df = pd.concat([evi, stemp], axis=1, keys=["evi", "stemp"])
# display(df.head(2))
df_year = df.groupby(df.index.year).mean()
df_year = df_year.interpolate()
# display(df_year.head(2))
df_res = df_year.unstack().reset_index()
df_res.columns = ["type", "buurt", "year", "value"]
df_res = df_res.pivot_table(
    index=["year", "buurt"], columns=["type"], values="value"
).reset_index()

In [34]:
df_total = df_res[df_res.year == 2019].sort_values("buurt")
df_eind = pd.merge(
    df_total,
    df_obs[df_obs.year == 2019][["obs", "wijk"]],
    left_on=["buurt"],
    right_on=["wijk"],
)[["buurt", "evi", "stemp", "obs"]].set_index("buurt")
df_eind.columns = ["evi", "stemp", "fauna_observaties"]
df_eind.head()

,evi,stemp,fauna_observaties
buurt,,,
Bavel,3194.347990,15.954720,2.0
Belcrum,1694.862223,18.132316,24.0
Biesdonk,2351.349178,18.027243,NaN
Blauwe Kei,2338.224492,16.862336,1.0
Boeimeer,2514.654895,16.768688,85.0


In [36]:
df_eind.to_hdf("./../data/dashboard-data.v01.hdf", "key")